## 크롤링 및 몽고 db 예제
* cine21 배우 랭킹 사이트 크롤링
 - 사이트 주소:http://www.cine21.com/rank/person/content
 - 요청 방식 확인 방법: 크롬 개발자 모드 -> Network -> content
    - http://www.cine21.com/rank/person/conten
    - Request Method: POST
    - section: actor
    - period_start: 2021-05
    - gender: all
    - page: 1

    

### 1. AWS ec2 서버에 도커로 실행되고 있는 mongodb에 접속
* 접속을 위한 정보 및 접속은 mongo_func 파일에 존재하는 함수로 구현
* 각 과정에 대한 접속정보를 확인

In [1]:
from bs4 import BeautifulSoup
import requests
from pymongo import mongo_client
import sys
import os
import time
from mongo_func import *
from datetime import datetime
import csv
# 시작 시간
start_time = time.time()
#ssh 접속 정보 가져오기 및 aws ec2 접속
ssh_conn_info = get_ssh_connect_information()
server = ssh_conn(ssh_conn_info[0],'ec2-user',ssh_conn_info[1],'127.0.0.1',27017)\

# 접속 상태 확인
if server is None:
    print("#### Error connecting to the AWS Server ####")
else:
    print("#### AWS Server connecting established ####")
    
server.start()

# 몽고db 접속 정보 가져오기 및 연결
mongo_conn_info = get_mongo_connect_information()
conn = mongo_client.MongoClient(f"mongodb://{mongo_conn_info[2]}:{mongo_conn_info[3]}@{mongo_conn_info[0]}:{mongo_conn_info[1]}")

# 접속 상태 확인
if conn is None:
    print("#### Error connecting to the MONGO ####")
else:
    print("#### MONGO connecting established ####")

#### AWS Server connecting established ####
#### MONGO connecting established ####


### 2. 접속 후 기본 데이터베이스 및 콜렉션 정보 설정

In [2]:
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

### 3. 크롤링 주소 requests

In [3]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2021-05'
post_data['gender'] = 'all'
post_data['page'] = '1'

res = requests.post(cine21_url,data=post_data)

#### 4. parsing 과 데이터 추출

In [4]:
soup = BeautifulSoup(res.content,"html.parser")

In [8]:
import re
actors = soup.select('li.people_li div.name')

actors_detail = list()

for j in actors:
    actor_link = 'http://www.cine21.com' + j.select_one("a").attrs['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_detail_info = default_info.select('li')

    actor_info_dict = dict()

    for k in actor_detail_info:
        actor_item_field = k.select_one('span.tit').text
        actor_itme_value = re.sub('<span.*?>.*?</span>','',str(k))
        actor_itme_value = re.sub('<.*?>','',actor_itme_value)
        actor_info_dict[actor_item_field] = actor_itme_value
    
    actors_detail.append(actor_info_dict)

In [10]:
actors_detail

[{'직업': '배우',
  '생년월일': '1968-01-21',
  '성별': '남',
  '신장/체중': '178cm, 68kg',
  '학교': '동의대학교 학사',
  '취미': '여행, 낚시',
  '소속사': '심엔터테인먼트'},
 {'다른 이름': '趙寅成',
  '직업': '배우',
  '생년월일': '1981-07-28',
  '성별': '남',
  '신장/체중': '186cm, 72kg',
  '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)',
  '취미': '모자모으기, 영화보기, 농구',
  '특기': '농구, 태권도(공인 4단)',
  '소속사': '싸이더스 HQ'},
 {'직업': '배우', '생년월일': '1983-00-00', '성별': '남'},
 {'직업': '배우', '생년월일': '1980-09-05', '성별': '남'},
 {'원어명': '許俊豪',
  '직업': '배우',
  '생년월일': '1964-03-03',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과',
  '취미': '만화책 보기',
  '특기': '야구, 농구',
  '소속사': '지티비엔터테인먼트'},
 {'직업': '배우',
  '생년월일': '1985-07-14',
  '성별': '남',
  '홈페이지': '\nhttps://twitter.com/masijacoke85\nhttps://www.instagram.com/masijacoke850714/\n',
  '신장/체중': '190cm',
  '소속사': '킹콩엔터테인먼트'},
 {'직업': '배우', '생년월일': '1978-03-03', '성별': '남', '신장/체중': '181cm, 72kg'}]

In [26]:
hits = soup.select('ul.num_info > li > strong')
movies = soup.select(('ul.mov_list'))
for index,actor in enumerate(actors):
    actor_name = re.sub('\(\w*\)','',actor.text)
    actor_hits = int(hits[index].text.replace(',',''))
    movie_titles = movies[index].select('li a span')
    movie_title_list = []
    for movie in movie_titles:
        movie_title_list.append(movie.text)   

배우이름:  김윤석
흥행지수:  31988
['모가디슈', '타짜', '화이 : 괴물을 삼킨 아이', '미성년', '1987']
배우이름:  조인성
흥행지수:  27991
['모가디슈', '클래식']
배우이름:  손석구
흥행지수:  27670
['범죄도시2', '연애 빠진 로맨스', '뺑반']
배우이름:  박지환
흥행지수:  24987
['유체이탈자', '해적: 도깨비 깃발', '범죄도시2', '지푸라기라도 잡고 싶은 짐승들', '봉오동 전투', '1987']
배우이름:  허준호
흥행지수:  23970
['모가디슈', '국가부도의 날']
배우이름:  이광수
흥행지수:  21742
['싱크홀', '해적: 도깨비 깃발', '해피 뉴 이어']
배우이름:  최귀화
흥행지수:  20219
['부기나이트', '기방도령', '범죄도시2', '원더풀 고스트', '택시운전사', '조작된 도시']


In [27]:
actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li > strong')
movies = soup.select(('ul.mov_list'))


actors_detail = list()
for index , j in enumerate(actors):
    
    actor_name = re.sub('\(\w*\)','',actor.text)
    actor_hits = int(hits[index].text.replace(',',''))
    movie_titles = movies[index].select('li a span')
    movie_title_list = []
    for movie in movie_titles:
        movie_title_list.append(movie.text)   
    movie_title_list

    actor_link = 'http://www.cine21.com' + j.select_one("a").attrs['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_detail_info = default_info.select('li')

    actor_info_dict = dict()

    actor_info_dict['배우이름'] = actor_name
    actor_info_dict['흥행지수'] = actor_hits
    actor_info_dict['출연영화'] = movie_title_list

    for k in actor_detail_info:
        actor_item_field = k.select_one('span.tit').text
        actor_itme_value = re.sub('<span.*?>.*?</span>','',str(k))
        actor_itme_value = re.sub('<.*?>','',actor_itme_value)
        actor_info_dict[actor_item_field] = actor_itme_value
    
    actors_detail.append(actor_info_dict)


actors_detail

[{'배우이름': '최귀화',
  '흥행지수': 31988,
  '출연영화': ['모가디슈', '타짜', '화이 : 괴물을 삼킨 아이', '미성년', '1987'],
  '직업': '배우',
  '생년월일': '1968-01-21',
  '성별': '남',
  '신장/체중': '178cm, 68kg',
  '학교': '동의대학교 학사',
  '취미': '여행, 낚시',
  '소속사': '심엔터테인먼트'},
 {'배우이름': '최귀화',
  '흥행지수': 27991,
  '출연영화': ['모가디슈', '클래식'],
  '다른 이름': '趙寅成',
  '직업': '배우',
  '생년월일': '1981-07-28',
  '성별': '남',
  '신장/체중': '186cm, 72kg',
  '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)',
  '취미': '모자모으기, 영화보기, 농구',
  '특기': '농구, 태권도(공인 4단)',
  '소속사': '싸이더스 HQ'},
 {'배우이름': '최귀화',
  '흥행지수': 27670,
  '출연영화': ['범죄도시2', '연애 빠진 로맨스', '뺑반'],
  '직업': '배우',
  '생년월일': '1983-00-00',
  '성별': '남'},
 {'배우이름': '최귀화',
  '흥행지수': 24987,
  '출연영화': ['유체이탈자',
   '해적: 도깨비 깃발',
   '범죄도시2',
   '지푸라기라도 잡고 싶은 짐승들',
   '봉오동 전투',
   '1987'],
  '직업': '배우',
  '생년월일': '1980-09-05',
  '성별': '남'},
 {'배우이름': '최귀화',
  '흥행지수': 23970,
  '출연영화': ['모가디슈', '국가부도의 날'],
  '원어명': '許俊豪',
  '직업': '배우',
  '생년월일': '1964-03-03',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과',
  '취미': '만화책

In [50]:

actors_detail = []

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2021-05'
post_data['gender'] = 'all'

for index in range(1,21):
    post_data['page'] = index
    
    res = requests.post(cine21_url,data=post_data)
    soup = BeautifulSoup(res.content,"html.parser")
    
    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select(('ul.mov_list'))
    rankings = soup.select('li.people_li > span.grade')

    for index , j in enumerate(actors):
        
        actor_name = re.sub('\(\w*\)','',j.text)
        actor_hits = int(hits[index].text.replace(',',''))
        movie_titles = movies[index].select('li a span')
        movie_title_list = []
        for movie in movie_titles:
            movie_title_list.append(movie.text)   
        movie_title_list

        actor_link = 'http://www.cine21.com' + j.select_one("a").attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_detail_info = default_info.select('li')

        actor_info_dict = dict()

        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_title_list
        actor_info_dict['랭킹'] = rankings[index].text

        for k in actor_detail_info:
            actor_item_field = k.select_one('span.tit').text
            actor_itme_value = re.sub('<span.*?>.*?</span>','',str(k))
            actor_itme_value = re.sub('<.*?>','',actor_itme_value)
            actor_info_dict[actor_item_field] = actor_itme_value
        
        actors_detail.append(actor_info_dict)

In [51]:
actors_detail

[{'배우이름': '김윤석',
  '흥행지수': 31988,
  '출연영화': ['모가디슈', '타짜', '화이 : 괴물을 삼킨 아이', '미성년', '1987'],
  '랭킹': '1',
  '직업': '배우',
  '생년월일': '1968-01-21',
  '성별': '남',
  '신장/체중': '178cm, 68kg',
  '학교': '동의대학교 학사',
  '취미': '여행, 낚시',
  '소속사': '심엔터테인먼트'},
 {'배우이름': '조인성',
  '흥행지수': 27991,
  '출연영화': ['모가디슈', '클래식'],
  '랭킹': '2',
  '다른 이름': '趙寅成',
  '직업': '배우',
  '생년월일': '1981-07-28',
  '성별': '남',
  '신장/체중': '186cm, 72kg',
  '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)',
  '취미': '모자모으기, 영화보기, 농구',
  '특기': '농구, 태권도(공인 4단)',
  '소속사': '싸이더스 HQ'},
 {'배우이름': '손석구',
  '흥행지수': 27670,
  '출연영화': ['범죄도시2', '연애 빠진 로맨스', '뺑반'],
  '랭킹': '3',
  '직업': '배우',
  '생년월일': '1983-00-00',
  '성별': '남'},
 {'배우이름': '박지환',
  '흥행지수': 24987,
  '출연영화': ['유체이탈자',
   '해적: 도깨비 깃발',
   '범죄도시2',
   '지푸라기라도 잡고 싶은 짐승들',
   '봉오동 전투',
   '1987'],
  '랭킹': '4',
  '직업': '배우',
  '생년월일': '1980-09-05',
  '성별': '남'},
 {'배우이름': '허준호',
  '흥행지수': 23970,
  '출연영화': ['모가디슈', '국가부도의 날'],
  '랭킹': '5',
  '원어명': '許俊豪',
  '직업': '배우',
  '생년월일': '1964-03-03',
  '성별': 

In [52]:
actor_collection.insert_many(actors_detail)

In [54]:
actor_collection.delete_many({})

In [2]:
conn.close()
server.close()

NameError: name 'conn' is not defined